In [1]:
import pandas as pd
import re

In [2]:
mitula = pd.read_csv("../datos_extraidos/casas_mitula.csv")

In [3]:
# Se eliminan todas las casas que tienen un remate bancario o hipotecario
# ya que no representan el valor real 
casas_remate = list(mitula[mitula.description.str.contains("remate|bancaria|recuperación|hipotecaria", case=False)].index)
mitula = mitula.drop(index=casas_remate)
mitula = mitula.drop(columns="publication_date")
mitula = mitula.reset_index(drop="first")

In [4]:
# Hay columnas con datos nulos
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          108 non-null    object 
 1   seller         108 non-null    object 
 2   property_type  108 non-null    object 
 3   address        108 non-null    object 
 4   price          108 non-null    int64  
 5   bedrooms       93 non-null     float64
 6   bathrooms      93 non-null     float64
 7   built_area     89 non-null     float64
 8   land_area      91 non-null     float64
 9   description    108 non-null    object 
dtypes: float64(4), int64(1), object(5)
memory usage: 8.6+ KB


In [5]:
# Al parecer todos los datos pertenecen a casas
mitula.property_type.unique()

array(['Casa', 'Terreno', 'Locales Comerciales', 'Villa',
       'Casas En Condominios', 'Edificio'], dtype=object)

In [6]:
# Se eliminan los datos nulos
mitula = mitula.dropna(ignore_index=True)

In [7]:
# Adecuo los valores de la columna adddress para hacer una búsqueda efectiva de los zip codes
mitula["address"] = mitula["address"].apply(lambda x: x.lower()\
                                                        .replace("á", "a")\
                                                        .replace("é", "e")\
                                                        .replace("í", "i")\
                                                        .replace("ó", "o")\
                                                        .replace("ú", "u"))
mitula["address"] = mitula["address"].str.replace("1ra secc", "1a secc")

### Creando la columna zip_code

In [8]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates(ignore_index=True)

# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [9]:
# Creo una función que asigna el zip code según la dirección
def buscador_zip_code(lista, diccionario):
    lista_coincidencias = []
    for i in lista:
        if (i.strip() == 'oaxaca'):
            pass
        else:
            regex = re.compile(fr"(.{{0,30}}{i.strip()}.{{0,30}})", re.IGNORECASE)
            coincidencia = list(filter(lambda x: regex.findall(x), diccionario.keys()))
            if len(coincidencia) > 0:
                lista_coincidencias.append(coincidencia[0])
    try:
        return diccionario["".join(lista_coincidencias[0])]
    except:
        None

In [10]:
mitula["zip_code"] = mitula["address"].apply(lambda x: x.split(","))
mitula["zip_code"] = mitula["zip_code"].apply(buscador_zip_code, diccionario=cp_dict)

In [11]:
# Existen dos filas con valores nulos en la columna zip_code
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          74 non-null     object 
 1   seller         74 non-null     object 
 2   property_type  74 non-null     object 
 3   address        74 non-null     object 
 4   price          74 non-null     int64  
 5   bedrooms       74 non-null     float64
 6   bathrooms      74 non-null     float64
 7   built_area     74 non-null     float64
 8   land_area      74 non-null     float64
 9   description    74 non-null     object 
 10  zip_code       72 non-null     float64
dtypes: float64(5), int64(1), object(5)
memory usage: 6.5+ KB


In [12]:
# Se eliminan los valores nulos
mitula = mitula.dropna(ignore_index=True)

### Creando la columna parking

In [13]:
# Con esta función localizo el texto donde especifica los lugares de estacionamiento en la descripción
def econtrar_textos(texto):
    regex = r"(?i)(.{0,10}estacionamientos?.{0,30}|.{0,10}cocheras?.{0,30}|.{0,10}garages?.{0,30}|.{0,10}parkings?.{0,30})"
    expresion = re.findall(regex, texto)
    expresion = "".join(expresion)
    if len(expresion) > 0:
        valor_de_celda = expresion
    else:
        valor_de_celda = "cero"
    return valor_de_celda

In [14]:
# Función para encontrar en la descripción el número de lugares de estacionamiento
def asignar_valor_estacionamiento(texto):
    # Buscar dígitos numéricos en el texto
    if texto != "cero":    
        numeros_encontrados = re.findall(r'\d+', texto)

        if numeros_encontrados:
            # Tomar el primer número encontrado y convertirlo a entero
            return int(numeros_encontrados[0])
        elif "para dos" in texto.lower():
            return 2
        elif "para tres" in texto.lower():
            return 3
        elif "para cuatro" in texto.lower():
            return 4
        elif "para cinco" in texto.lower():
            return 5
        else:
            return 1

In [15]:
# Aplicar las funciones para buscar los valores de parking
mitula["parking"] = mitula["description"].apply(econtrar_textos)
mitula["parking"] = mitula["parking"].apply(asignar_valor_estacionamiento)

In [16]:
# Se tienen algunos valores nulos en la columna parking
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          72 non-null     object 
 1   seller         72 non-null     object 
 2   property_type  72 non-null     object 
 3   address        72 non-null     object 
 4   price          72 non-null     int64  
 5   bedrooms       72 non-null     float64
 6   bathrooms      72 non-null     float64
 7   built_area     72 non-null     float64
 8   land_area      72 non-null     float64
 9   description    72 non-null     object 
 10  zip_code       72 non-null     float64
 11  parking        49 non-null     float64
dtypes: float64(6), int64(1), object(5)
memory usage: 6.9+ KB


In [17]:
# Una de las propiedades es un conjunto de departamentos
# y no corresponde a la misma naturaleza que el resto de casas
mitula.sort_values(by="land_area", ascending=False).head(5)

,title,seller,property_type,address,price,bedrooms,bathrooms,built_area,land_area,description,zip_code,parking
53,ATENCION INVERSIONISTAS - (3),Mitula,Casa,"oaxaca, oaxaca, benito juarez",13000000,1.0,1.0,890.0,1225.0,Atención inversionistasRemax Cantera propone c...,68030.0,NaN
4,Casa en Venta de un Nivel 2 Recamaras en San F...,Mitula,Casa,"piedra de sal, agencia municipal de san felipe...",7200000,2.0,1.0,150.0,938.0,Casa habitación rustica campirana en un solo n...,68276.0,NaN
54,PROPIEDAD CON 6 DEPARTAMENTOS Y CASA - (3),Mitula,Casa,"oaxaca, oaxaca, reforma",30000000,10.0,10.0,760.0,839.0,PROPIEDAD MUY BIEN UBICADA CON 6 DEPARTAMENTOS...,68050.0,1.0
62,CASA EN VENTA CON LOCALES COMERCIALES - (3),Mitula,Casa,"oaxaca, oaxaca, oaxaca centro",20450000,5.0,4.0,407.0,545.0,"CASA HABITACION CON DISTRIBUIDOR, DESAYUNADOR,...",68000.0,NaN
44,CASA EN VENTA CON LOCALES COMERCIALES - (3),Mitula,Casa,"oaxaca, oaxaca, oaxaca centro",20450000,5.0,4.0,407.0,545.0,"CASA HABITACION CON DISTRIBUIDOR, DESAYUNADOR,...",68000.0,NaN


In [18]:
# Se elimina la propiedad que no corresponde a una casa
# sino a un conjunto de locales
no_casa = list(mitula[mitula["title"].str.contains("propiedad con 6 departamentos y casa", case=False)].index)
mitula = mitula.drop(index=no_casa)
mitula = mitula.drop_duplicates()
mitula = mitula.reset_index(drop="first")

In [19]:
# Se corrijen los valores de las columnas bedrooms y bathrooms de la fila con 
# el valor de land_area más grande ya que la información que aparece en las columnas
# no se corresponde con su descripción
max_land_area = list(mitula.query("price==13000000 & land_area==1225.0").index)
mitula.loc[max_land_area, "bedrooms"] = 10
mitula.loc[max_land_area, "bathrooms"] = 9

In [20]:
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          62 non-null     object 
 1   seller         62 non-null     object 
 2   property_type  62 non-null     object 
 3   address        62 non-null     object 
 4   price          62 non-null     int64  
 5   bedrooms       62 non-null     float64
 6   bathrooms      62 non-null     float64
 7   built_area     62 non-null     float64
 8   land_area      62 non-null     float64
 9   description    62 non-null     object 
 10  zip_code       62 non-null     float64
 11  parking        43 non-null     float64
dtypes: float64(6), int64(1), object(5)
memory usage: 5.9+ KB


In [21]:
mitula.to_csv("datos_mitula.csv", index=False)